<a href="https://colab.research.google.com/github/bradshimmin/llm_experiments/blob/main/OpenAI_Structured_Outputs_for_AI_Skills_tiktok_30aug24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
# Set up software
#
!pip -q install openai tiktoken


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.9 MB/s eta 0:00:00


In [50]:
# Import libraries
#
import os
import openai
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import textwrap
import tiktoken

from enum import Enum
from typing import Union
from pydantic import BaseModel
from openai import OpenAI

print("all done")

all done


In [3]:
# Load openai key
#
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OpenAI_key')

---
# Set up AI Skills data

In [4]:
# Mount Google Drive
#
from google.colab import drive
drive.mount('/content/gdrive')

print("all done")

Mounted at /content/gdrive
all done


In [5]:
# Load our sample dataset using 2q24 cleaned df
#
df_sample = pd.read_pickle("/content/gdrive/MyDrive/Colab Notebooks/Datasets/ai_skills_tracker/ai_skills_2q24/ai_skills_2q24_final_clean_28aug24.pkl")


In [6]:
df_sample.shape

(205517, 31)

In [7]:
df_sample.head(n=2)

,RecordID,Job title,Full job description,Date collected,Date posted,Week,Year,Month,Quarter,Hiring entity,...,Salary start,Salary end,Main industry,Job title category,AI technologies,AI skills,Source name,Source URL,File Date,Non-ASCII Chars
0,1,Data Engineer,"""ALL our jobs are US based and candidates must...",2023-06-11,2023-06-11,23,2023,6,2023Q2,TechFetch.com - On Demand Tech Workforce hirin...,...,NaN,NaN,IT Services,Statistician Data Science,"Bachelor, Ab Initio, Big Data, Healthcare Insu...","Databases, Computer Science, Information Techn...",LinkedIn,https://www.linkedin.com/jobs/view/data-engine...,2023-04-01,[]
1,2,Data Scientist,"""ALL our jobs are US based and candidates must...",2023-06-11,2023-06-11,23,2023,6,2023Q2,TechFetch.com - On Demand Tech Workforce hirin...,...,NaN,NaN,IT Services,Statistician AI,"Transformation, Sas, Azure, Databricks, Data L...","Support, Business Intelligence, Server, Langua...",LinkedIn,https://www.linkedin.com/jobs/view/data-scient...,2023-04-01,[]


In [19]:
# Pull out tiktok data
# Step 1: Filter by "Hiring entity" to include only "TikTok"
tiktok_jobs = df_sample[df_sample['Hiring entity'].str.contains("TikTok", case=False, na=False)]
print(f'TikTok jobs: {len(tiktok_jobs)}')

TikTok jobs: 1640


In [20]:
# prompt: remove duplicates based on "Full job description"
#
tiktok_jobs = tiktok_jobs.drop_duplicates(subset=['Full job description'])
print(f'Deduped TikTok jobs: {len(tiktok_jobs)}')

Deduped TikTok jobs: 1633


In [21]:
# prompt: Print the number of jobs per month within tiktok_jobs using "Date posted"
#
tiktok_jobs['Date posted'] = pd.to_datetime(tiktok_jobs['Date posted'])
tiktok_jobs_per_month = tiktok_jobs.groupby(pd.Grouper(key='Date posted', freq='M')).size()
print(tiktok_jobs_per_month)


Date posted
2022-02-28      3
2022-03-31     19
2022-04-30     27
2022-05-31     13
2022-06-30      4
2022-07-31     29
2022-08-31     45
2022-09-30     84
2022-10-31    124
2022-11-30     35
2022-12-31     98
2023-01-31     40
2023-02-28     51
2023-03-31     97
2023-04-30     57
2023-05-31     54
2023-06-30      6
2023-07-31     83
2023-08-31    101
2023-09-30     78
2023-10-31    135
2023-11-30     89
2023-12-31     40
2024-01-31    137
2024-02-29     45
2024-03-31     22
2024-04-30     48
2024-05-31     58
2024-06-30     11
Freq: M, dtype: int64


In [22]:
# Filter jobs posted in the last 3 months (given that March is the last collection date)
#
date_range = datetime.now() - timedelta(days=182)
recent_tiktok_jobs = tiktok_jobs[tiktok_jobs['Date posted'] >= date_range]

recent_tiktok_jobs.shape

(139, 31)

In [25]:
# Check date range
print(recent_tiktok_jobs['Date posted'].min())
print(recent_tiktok_jobs['Date posted'].max())


2024-03-02 00:00:00
2024-06-17 00:00:00


In [26]:
# prompt: Modify print() output so that the text wraps automatically at 250 charcters
def print_wrapped(text, width=100):
  wrapped_text = textwrap.fill(text, width=width)
  return wrapped_text


In [28]:
job_post = recent_tiktok_jobs['Full job description'].iloc[0]
print_wrapped(job_post)

"Responsibilities TikTok is the leading destination for short-form mobile video. At TikTok, our\nmission is to inspire creativity and bring joy. TikTok's global headquarters are in Los Angeles and\nSingapore, and its offices include New York, London, Dublin, Paris, Berlin, Dubai, Jakarta, Seoul,\nand Tokyo. Why Join Us Creation is the core of TikTok's purpose. Our platform is built to help\nimaginations thrive. This is doubly true of the teams that make TikTok possible. Together, we\ninspire creativity and bring joy - a mission we all believe in and aim towards achieving every day.\nTo us, every challenge, no matter how difficult, is an opportunity; to learn, to innovate, and to\ngrow as one team. Status quo? Never. Courage? Always. At TikTok, we create together and grow\ntogether. That's how we drive impact - for ourselves, our company, and the communities we serve.\nJoin us. Our Trust and Safety team is fast growing and responsible for building machine learning\nmodels and systems to

---
# Test OpenAI and Pydantic


In [29]:
# Set up a new, simplified Pydantic class
#
class Title(BaseModel):
    """
    Represents the full title of the job posting.

    This model captures the full title of the job posting
    Examples include:
    'Senior Machine Learning Engineer, TikTok Ads Core- Business Account',
    'Lead Research Engineer, Machine Learning - TikTok Privacy Innovation Lab'
    """
    title: str # Added type annotation for title

class Role(BaseModel):
    """
    Represents the area in which the employee will work.

    This model capture the area, department, or division in which the employee
    will work within the company. Examples include:
    'Research and development', 'User engagement', 'Systems infrastructure',
    'Trust and safety'
    """
    role: str # Added type annotation for role

class Objectives(BaseModel):
    """
    Represents the stated objectives the company has set for the employee.
    Examples include:
    ' 1. Drive insightful understanding of Live users and creators to accelerate
    better user, creator experience in Live.', '2. Keep track of core metrics and
    attribute metric changes to root causes.', '3. Use data and\nexperiments to
    verify hypotheses around bottlenecks in product growth, and consult on the
    direction for improvement.', '4. Work with MLE and product stakeholders to
    deLiver product insights and strategy.'
    """
    objectives: list[str] # Added type annotation for objectives

class Technologies(BaseModel):
    """
    Represents a technology-based skills required by a hiring entity.

    This model captures basic information about a products and technologies
    mentioned within this job post. Examples include:
    'Spark', 'Hadoop', 'PyTorch', 'Git', 'OpenAI', 'Vertex AI', 'Google Cloud',
    'Python', 'SQL', 'Llama', 'LangChain'
    """
    technologies: list[str]  # Added type annotation for technologies

class Skills(BaseModel):
    """
    Represents a technology-based skills required by a hiring entity.

    This model captures basic information about a business skills mentioned
    within this job post. Examples include:
    'data collection', 'data cleaning', 'data analysis', 'reporting',
    'data visualization', 'training', 'stakeholder collaboration', 'MLOps',
    'DataOps', 'Responsible AI', 'Data Engineering', 'Data Science', 'Data Warehousing'
    """
    skills: list[str]  # Added type annotation for skills

class Summary(BaseModel):
    """
    Represents the area in which the employee will work.

    This model capture the a concise, 250 word summary of the job posting, focusing
    on the job itself rather than the company or the job requirements and benefits.'
    """
    summary: str # Added type annotation for role

class JobResponseSimple(BaseModel):
    """
    Represents the structured response for an article analysis.

    This model aggregates information about title, role, objectives, technologies,
    and skills mentioned in a job post
    """
    title: str
    role: str
    objectives: list[Objectives]
    technologies: list[Technologies]
    skills: list[Skills]
    summary: str

In [30]:
# Simplify our inference function
#
client = OpenAI()

title = ""
role = ""
objectives = []
technologies = []
skills = []
summary = ""

def parse_post_simple(jobpost):
    completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
            {"role": "system", "content": "You are a helpful assistant that \
            scans job posts for organization info, the technologies and products, \
            and business skills required for a given job posting. Please be thorough \
            and include as many technologies and skills as you can in your response. \
            Do not include the examples UNLESS they actual appear in the jobpost. \
            And remember, this is an analysis, so be detailed and concise, and \
            above all else, do not hallucinate."},
            {"role": "user", "content": jobpost},
        ],
        response_format=JobResponseSimple,
    )

    message = completion.choices[0].message
    if message.parsed:
        output = []
        output.append(str(message.parsed.title))
        output.append(str(message.parsed.role))
        for objectives in message.parsed.objectives:
            output.append(str(objectives))
        for technologies in message.parsed.technologies:
            output.append(str(technologies))
        for skills in message.parsed.skills:
            output.append(str(skills))
        output.append(str(message.parsed.summary))
        return output
    else:
        return message.refusal

In [31]:
# Run our test
#
test_ner = parse_post_simple(job_post)
print(test_ner)

['Machine Learning Engineer', 'Machine Learning Engineer', 'objectives=[\'Design and develop advanced machine learning solutions\', "Deploy machine learning models to TikTok\'s global platform", \'Collaborate with product teams to define business objectives and improve trust and safety strategy\', \'Collaborate with engineering teams to deploy machine learning models, design online model workflow, and implement serving pipelines at scale\', \'Collaborate with data scientists to understand key challenges and propose data-driven strategies\']', "technologies=['Machine Learning', 'Computer Vision', 'Natural Language Processing', 'Large Language Models', 'Active Learning', 'Reinforcement Learning', 'Recommendation Systems']", "skills=['Strong coding skills in Python', 'Software engineering', 'Communication and teamwork skills', 'Machine learning expertise in CV, NLP, and multimodal areas', 'Data-driven strategy development']", 'TikTok is looking for a Machine Learning Engineer to develop a

---
# Append data to individual columns in our dataframe


In [32]:
# Inference function definition
#
title = ""
role = ""
objectives = []
technologies = []
skills = []
summary = ""

def parse_post_simple(jobpost):
    completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
            {"role": "system", "content": "You are a helpful assistant that \
            scans job posts for organization info, the technologies and products, \
            and business skills required for a given job posting. Please be thorough \
            and include as many technologies and skills as you can in your response. \
            Do not include the examples UNLESS they actual appear in the jobpost. \
            And remember, this is an analysis, so be detailed and concise, and \
            above all else, do not hallucinate."},
            {"role": "user", "content": jobpost},
        ],
        response_format=JobResponseSimple,
    )

    message = completion.choices[0].message
    if message.parsed:
        # Separate lists for technologies and business skills
        title = str(message.parsed.title)
        role = str(message.parsed.role)
        objectives = [str(obj) for obj in message.parsed.objectives]
        technologies = [str(tech) for tech in message.parsed.technologies]
        skills = [str(skill) for skill in message.parsed.skills]
        summary = str(message.parsed.summary)

        # Return as separate values
        return title, role, objectives, technologies, skills, summary
    else:
        return "", "", [], [], [], ""  # Return empty lists in case of refusal or no parsing


In [33]:
# Convert sample_jobs into df for processing
#
df = recent_tiktok_jobs.copy()
df.shape

(139, 31)

In [35]:
# Apply the parse_post_simple function to each job post
#
df['title'], df['role'], df['objectives'], df['technologies'], df['skills'], df['summary'] \
= zip(*df['Full job description'].apply(parse_post_simple))


In [36]:
df.columns

Index(['RecordID', 'Job title', 'Full job description', 'Date collected',
       'Date posted', 'Week', 'Year', 'Month', 'Quarter', 'Hiring entity',
       'Entity size', 'Home office hybrid work', 'Full or part time status',
       'Number of applicants', 'Longitude', 'Latitude', 'City', 'Territory',
       'Region', 'Experience level', 'Education level', 'Salary start',
       'Salary end', 'Main industry', 'Job title category', 'AI technologies',
       'AI skills', 'Source name', 'Source URL', 'File Date',
       'Non-ASCII Chars', 'title', 'role', 'objectives', 'technologies',
       'skills', 'summary'],
      dtype='object')

In [37]:
# Check results
#
def check_results(df, record):
    """Check the results of the parse_post_simple function."""
    print(f"title: {df.title.iloc[record]}")
    print(f"role: {df.role.iloc[record]}")
    print(f"objectives: {df.objectives.iloc[record]}")
    print(f"technologies: {df.technologies.iloc[record]}")
    print(f"skills: {df.skills.iloc[record]}")
    print(f"summary: {df.summary.iloc[record]}")


In [39]:
check_results(df, 4)


title: Data Scientist - Business Risk Integrated Control
role: Data Scientist
objectives: ["objectives=['Build rules, algorithms and machine learning models to respond to and mitigate business risks in TikTok products and platforms.', 'Analyze business and security data, uncover evolving attack motions, identify weaknesses and opportunities in risk defense solutions.', 'Define risk control measurements, quantify, generalize and monitor risk-related business and operational metrics.', 'Align risk teams and their stakeholders on risk control numeric goals, promote impact-oriented, data-driven data science practices for risks.']"]
technologies: ["technologies=['SQL', 'R', 'Python', 'Machine Learning', 'Statistical Analytical Tools']"]
skills: ["skills=['Data Science', 'Statistical Analysis', 'Critical Thinking', 'Risk Analysis', 'Data-Driven Decision Making', 'Operational Metrics Monitoring']"]
summary: The Data Scientist role in the Business Risk Integrated Control (BRIC) team at TikTok 

In [40]:
# Drop 'unwanted columns
#
df = df.drop(['Job title category', 'AI technologies', 'AI skills', 'Non-ASCII Chars'], axis=1)
df.columns

Index(['RecordID', 'Job title', 'Full job description', 'Date collected',
       'Date posted', 'Week', 'Year', 'Month', 'Quarter', 'Hiring entity',
       'Entity size', 'Home office hybrid work', 'Full or part time status',
       'Number of applicants', 'Longitude', 'Latitude', 'City', 'Territory',
       'Region', 'Experience level', 'Education level', 'Salary start',
       'Salary end', 'Main industry', 'Source name', 'Source URL', 'File Date',
       'title', 'role', 'objectives', 'technologies', 'skills', 'summary'],
      dtype='object')

In [41]:
# Write our new df to disk
#
df.to_pickle("/content/gdrive/MyDrive/Colab Notebooks/Datasets/df_ner_tiktok_30aug24.pkl")
df.to_csv("/content/gdrive/MyDrive/Colab Notebooks/Datasets/df_ner_tiktok_30aug24.csv")

---
# Summarizing and analyzing our full list of jobs

In [42]:
# Combine our test data into a single block of text
#
job_data = "\n\n".join([
    f"Title: {row['title']}\n"
    f"Date posted: {row['Date posted']}\n"
    f"Role: {row['role']}\n"
    f"Objectives: {row['objectives']}\n"
    f"Technologies: {row['technologies']}\n"
    f"Skills: {row['skills']}\n"
    f"Summary: {row['summary']}\n"
    for index, row in df.iterrows()
])
print("all done")

all done


In [46]:
# prompt: print the first few lines of text from job_data
#
print(job_data[:500]) # Print first 500 characters


Title: Machine Learning Engineer
Date posted: 2024-03-15 00:00:00
Role: Machine Learning Engineer - Trust and Safety
Objectives: ["objectives=['Design and develop advanced machine learning solutions to protect users from negative content.', 'Collaborate with product teams to define business objectives for trust and safety.', 'Deploy machine learning models and implement serving pipelines at scale.', 'Identify challenges and propose data-driven solutions in content moderation.']"]
Technologies: [


In [47]:
# Generate prompt with job data
#
system_prompt = f"""
You are an AI assistant specializing in analyzing job data to understand a company's strategic focus and workforce priorities.
Given multiple job postings with roles, objectives, technologies, skills, and summaries, your task is to generate a comprehensive markdown summary
detailing the overall nature of the job roles and what the company is focusing on. And remember, this is an analysis, so be detailed and concise,
and above all else, do not hallucinate.

Use the following outline to structure the analysis:

1. **Introduction**
   Provide an introduction about TikTok’s current focus and strategic areas based on the job data analyzed.

2. **Job Roles and Objectives**
   Summarize the main job roles within the data set, detailing the key objectives of these roles. Highlight common themes and areas of strategic emphasis.

3. **Technologies and Tools**
   Identify the technologies and tools most frequently mentioned across the job posts. Explain how these technologies contribute to TikTok’s overall strategy.

4. **Skills in Demand**
   Outline the critical skills that TikTok seeks in its employees, both technical and soft skills. Discuss how these skills align with the company’s objectives.

5. **Strategic Focus Areas**
   Discuss what the job data suggests about TikTok’s strategic focus.

6. **Evolution of TikTok AI staffing practices**
   Provide an analysis of the way TikTok's AI professional hiring practices have changed, highlighting any emerging trends or notable observations.

7. **Conclusion**
   Provide a conclusion summarizing TikTok’s overarching goals based on the job data, highlighting any emerging trends or notable observations.

**Output Format**: Provide a markdown-formatted analysis based on the data.
"""


In [51]:
# Calculate the number of tokens we'll be sending to the model in our job_data prompt
#
# Choose the appropriate model's tokenizer
encoding = tiktoken.encoding_for_model("gpt-4o-mini")  # or whatever mode model

# Encode the prompt to get the list of tokens
tokens = encoding.encode(system_prompt + job_data)

# Calculate the number of tokens
num_tokens = len(tokens)

print(f"Number of tokens in the prompt: {num_tokens}")


Number of tokens in the prompt: 34823


# This is way too big. Mmmm. Will try something directly in their playground

In [52]:
# prompt: save job_data to a text file
#
with open('job_data_30aug24.txt', 'w') as f:
  f.write(job_data)
